# Self Driving Traffic Sign Detection

Isaiah Jenkins

## Import the required libraries

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Available devices:", tf.config.list_physical_devices())
import pandas as pd
import os
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array


TensorFlow version: 2.16.2
Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## 1. About the data

1. a. Description

Throughout this analysis we will explore Udacity's Self Driving dataset. This dataset consists of images for thousands of pedestrians, bikers, cars, and traffic lights. Although traffic light images are underrepresented in the dataset, the focus of the analysis will be based solely on traffic light images.

1. b. Data dictionary, 97,942 labels across 11 classes and 15,000 images, 1,720 null examples (images with no labels).

Class Balance across images (labels in each image)

* car - 64,399 - over represented
* pedestrian - 10,806
* trafficLight-Red - 6,870
* trafficLight-Green - 5,465 - under represented
* truck - 3,623 - under represented
* trafficLight - 2,568 - under represented
* biker - 1,864 - under represented
* trafficLight-RedLeft - 1,751 - under represented
* trafficLight-GreenLeft - 310 - under represented
* trafficLight-Yellow - 272 - under represented
* trafficLight-YellowLeft - 14 - under represented

## 2. Objectives

Throughout this analysis, we will explore and build various deep learning convolutional neural network (CNN) architectures to detect traffic light signs, aiming to optimize accuracy and efficiency. Our objective is to compare different model variations, such as CNNs with different depths, pre-trained models, and data augmentation techniques, to determine the most effective approach. Potential challenges include handling variations in lighting conditions, occlusions, and small object sizes, which may impact detection performance. Additionally, dataset imbalances and misclassifications due to similar-looking traffic signs could introduce biases, requiring careful preprocessing and model tuning.

## 3. Data Exploration, Cleaning and Feature Engineering

* Extract traffic sign & traffic light images from the dataset to help with computational efficiency.
* Resize images to standardize dataset making it computationally efficient making it easier to analyze data.
* Normalize pixel values (0-1 range) to help with generalization.
* Augment data (rotation, brightness shifts, contrast changes) to improve generalization.

### Load in data

In [2]:
DATASET_PATH = 'data/export'
CSV_PATH = os.path.join(DATASET_PATH, '_annotations.csv')

In [3]:
df = pd.read_csv(CSV_PATH) # load annotations

In [4]:
file_names = df['filename'].values # image files names
labels = df['class'].values # classes

In [5]:
label_map = {"trafficLight-Red": 0, "trafficLight-RedLeft": 1, "trafficLight_Yellow": 2, "trafficLight_YellowLeft": 3, "trafficLight-Green": 4, "trafficLight-GreenLeft": 5 }  # Adjust based on dataset
labels = [label_map[label] for label in labels if label in label_map]  # Convert text labels to integers
labels = to_categorical(labels, num_classes=6)  # Convert to one-hot encoding

In [6]:
IMG_SIZE = (224, 224)

In [7]:
# Function to Load & Preprocess Images
def load_and_preprocess_image(file_name, label):
    img_path = os.path.join(DATASET_PATH, file_name)  # Update image folder path
    img = load_img(img_path, target_size=IMG_SIZE)  # Load & Resize Image
    img = img_to_array(img) / 255.0  # Convert to NumPy array & Normalize (0-1)
    return img, label

In [8]:
# Load All Images
image_data = [load_and_preprocess_image(f, l) for f, l in zip(file_names, labels)]

In [9]:
# Convert to TensorFlow Dataset
images, labels = zip(*image_data)
images = tf.convert_to_tensor(images, dtype=tf.float32)
labels = tf.convert_to_tensor(labels, dtype=tf.float32)

2025-02-12 09:31:56.972049: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-02-12 09:31:56.972090: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-02-12 09:31:56.972104: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-02-12 09:31:56.972276: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-12 09:31:56.972296: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [10]:
# Create a tf.data Dataset for Training
dataset = tf.data.Dataset.from_tensor_slices((images, labels)).batch(32).shuffle(1000)

ValueError: Dimensions 224 and 6 are not compatible

## 4. CNN Models

### Summary of Models

In [ ]:
# Preview Dataset
for img_batch, label_batch in dataset.take(1):
    print(f"Image Batch Shape: {img_batch.shape}")
    print(f"Label Batch Shape: {label_batch.shape}")

## 5. Insights and Key Findings

## 6. Next Steps